In [9]:
# 1) Imports e configuração básica
from pathlib import Path
import pandas as pd

In [10]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# 2) Caminhos do projeto
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data"

# 3) Leitura dos CSVs
results = pd.read_csv(DATA_RAW / "results.csv", parse_dates=["date"])
goalscorers = pd.read_csv(DATA_RAW / "goalscorers.csv", parse_dates=["date"])
shootouts = pd.read_csv(DATA_RAW / "shootouts.csv", parse_dates=["date"])

# 4) Checagens rápidas
print("results:", results.shape)
print("goalscorers:", goalscorers.shape)
print("shootouts:", shootouts.shape)

display(results.head)
display(goalscorers.head(4))
display(shootouts.head(4))

# 5) Validação mínima de colunas-chave (evita erro lá na frente)
# required_results = {"date", "home_team", "away_team", "home_score", "away_score", "neutral"}
# missing = required_results - set(results.columns)
# if missing:
# raise ValueError(f"results.csv está sem colunas esperadas: {missing}")

results: (44934, 9)
goalscorers: (41113, 8)
shootouts: (558, 4)


<bound method NDFrame.head of             date    home_team away_team  home_score  away_score               tournament       city      country  \
0     1872-11-30     Scotland   England           0           0                 Friendly    Glasgow     Scotland   
1     1873-03-08      England  Scotland           4           2                 Friendly     London      England   
2     1874-03-07     Scotland   England           2           1                 Friendly    Glasgow     Scotland   
3     1875-03-06      England  Scotland           2           2                 Friendly     London      England   
4     1876-03-04     Scotland   England           3           0                 Friendly    Glasgow     Scotland   
...          ...          ...       ...         ...         ...                      ...        ...          ...   
44929 2023-09-12      Belgium   Estonia           5           0  UEFA Euro qualification   Brussels      Belgium   
44930 2023-09-12       Sweden   Austria   

,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False


,date,home_team,away_team,winner
0,1967-08-22,India,Taiwan,Taiwan
1,1971-11-14,South Korea,Vietnam Republic,South Korea
2,1972-05-07,South Korea,Iraq,Iraq
3,1972-05-17,Thailand,South Korea,South Korea


In [11]:
# 6) Período dos dados
inicio = results["date"].min().strftime("%d/%m/%Y")
fim = results["date"].max().strftime("%d/%m/%Y")

print(f"Período dos dados: {inicio} → {fim}")

Período dos dados: 30/11/1872 → 12/09/2023


In [12]:
wc = results.query("tournament == 'FIFA World Cup'").copy()

print("Jogos de Copa do Mundo:", wc.shape[0])
print("Copas:", wc["date"].dt.year.unique())


Jogos de Copa do Mundo: 964
Copas: [1930 1934 1938 1950 1954 1958 1962 1966 1970 1974 1978 1982 1986 1990
 1994 1998 2002 2006 2010 2014 2018 2022]


In [13]:
# O futebol era mais ofensivo?
wc["total_gols"] = wc["home_score"] + wc["away_score"]

In [14]:
wc["copa"] = wc["date"].dt.year
wc[["date", "copa"]].drop_duplicates().sort_values("copa").head(10)

,date,copa
1311,1930-07-13,1930
1313,1930-07-14,1930
1315,1930-07-15,1930
1316,1930-07-16,1930
1317,1930-07-17,1930
1320,1930-07-18,1930
1321,1930-07-19,1930
1323,1930-07-20,1930
1325,1930-07-21,1930
1326,1930-07-22,1930


Ao analisar a média de gols por partida nas edições da Copa do Mundo, observa-se uma tendência de redução ao longo do tempo, com Copas mais recentes apresentando valores mais baixos e estáveis quando comparadas às edições iniciais.

In [15]:
baseline = (
    wc.groupby("copa")
      .agg(
          jogos=("total_gols", "size"),
          total_gols=("total_gols", "sum"),
          media_gols=("total_gols", "mean")
      )
      .reset_index()
)

display(baseline.round(2))


,copa,jogos,total_gols,media_gols
0,1930,18,70,3.89
1,1934,17,70,4.12
2,1938,18,84,4.67
3,1950,22,88,4.00
4,1954,26,140,5.38
5,1958,35,126,3.60
6,1962,32,89,2.78
7,1966,32,89,2.78
8,1970,32,95,2.97
9,1974,38,97,2.55


**Refinamento da análise: média vs mediana**

A média oferece uma visão inicial da tendência ao longo do tempo, mas não representa necessariamente o jogo típico. Para complementar essa análise, a mediana de gols por partida foi incluída.

In [16]:
refino = (
    wc.groupby("copa")
      .agg(
          media_gols=("total_gols", "mean"),
          mediana_gols=("total_gols", "median")
      )
      .reset_index()
)

display(refino.round(2))


,copa,media_gols,mediana_gols
0,1930,3.89,4.0
1,1934,4.12,4.0
2,1938,4.67,4.0
3,1950,4.00,4.0
4,1954,5.38,5.0
5,1958,3.60,3.0
6,1962,2.78,3.0
7,1966,2.78,3.0
8,1970,2.97,3.0
9,1974,2.55,2.0


**Conclusão:** A redução observada na média de gols é acompanhada por uma queda da mediana, indicando que o jogo típico das Copas mais recentes apresenta menor número de gols em comparação às edições antigas.